In [24]:
!pip install openpyxl

                                              0.0/250.0 kB ? eta -:--:--
     ------------------------------------- 250.0/250.0 kB 15.0 MB/s eta 0:00:00


In [26]:
import pandas as pd
import numpy as np
import re
import os

In [27]:
naver_df = pd.read_csv('../../naver_crw/DAOU/test_large_out_v3.csv', index_col=0, encoding='euc-kr')

In [33]:
print(naver_df.info())
naver_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6664 entries, 0 to 6663
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   사업장명                    6664 non-null   object 
 1   업소유형                    874 non-null    object 
 2   업태구분명                   6664 non-null   object 
 3   지번주소                    6653 non-null   object 
 4   도로명주소                   3986 non-null   object 
 5   가까운 지하철역                5774 non-null   object 
 6   지하철역과의 거리               6664 non-null   float64
 7   도보시간                    6664 non-null   float64
 8   위도                      5774 non-null   float64
 9   경도                      5774 non-null   float64
 10  별점                      310 non-null    float64
 11  방문자 리뷰수                 685 non-null    object 
 12  블로그 리뷰수                 513 non-null    object 
 13  한실수                     5434 non-null   float64
 14  양실수                     5347 non-null   

,사업장명,업소유형,업태구분명,지번주소,도로명주소,가까운 지하철역,지하철역과의 거리,도보시간,위도,경도,...,네이버 이런점이 좋아요 총합,네이버 이런점이 좋아요 {항목:좋아요수},데이터갱신일자,인허가일자,인허가취소일자,영업상태코드,폐업일자,휴업시작일자,휴업종료일자,재개업일자
0,어반스테이 명동,레지던스,숙박업(생활),서울특별시 중구 인현동1가 158,"서울특별시 중구 충무로 24, 3~17층 (인현동1가)",충무로,223.010258,3.343482,37.563358,126.993360,...,41.0,"{'깨끗해요': 29},{'조용히 쉬기 좋아요': 22},{'냉난방이 잘돼요': 2...",2022-12-03 00:03:00.0,2021-01-15,NaN,1,NaN,NaN,NaN,NaN
1,오요5(oYo5),호스텔,관광호텔,서울특별시 중구 남산동2가 35-2,"서울특별시 중구 퇴계로18길 43, 지층, 1~2층 (남산동2가)",명동,263.215774,3.946263,37.558934,126.984838,...,NaN,NaN,2022-12-03 00:03:00.0,2019-08-02,NaN,1,NaN,NaN,NaN,NaN
2,더스테이클래식호텔,호텔,숙박업(생활),서울특별시 중구 남대문로3가 94,서울특별시 중구 남대문로 27 (남대문로3가),회현(남대문시장),330.151983,4.949805,37.561480,126.978525,...,NaN,NaN,2022-12-02 23:05:00.0,2023-03-13,NaN,1,NaN,NaN,NaN,NaN
3,오요5롯지A(oYo5LodgeA),NaN,숙박업 기타,서울특별시 중구 남산동2가 33-3 1~2층,"서울특별시 중구 퇴계로18길 41 (남산동2가, 1~2층)",명동,240.223310,3.601549,37.559108,126.984981,...,NaN,NaN,2022-12-03 00:03:00.0,2017-04-26,NaN,1,NaN,NaN,NaN,NaN
4,오요5롯지B(oYo5LodgeB),NaN,관광호텔,"서울특별시 중구 남산동2가 33 1,2층","서울특별시 중구 퇴계로18길 37 (남산동2가, 1,2층)",명동,226.602301,3.397336,37.559222,126.985041,...,NaN,NaN,2022-12-03 00:03:00.0,2017-04-26,NaN,1,NaN,NaN,NaN,NaN


In [32]:
yanolja_df = pd.read_excel('../DABA/middle2.xlsx')
yanolja_df.head()

,이름,종류,평점,숙박가격,도보시간,인기테그,별점 높은순 후기,별점 낮은순 후기,주소,객실수,총후기,대실가격,대실시간,위도,경도
0,앰배서더 서울 풀만 호텔,프리미엄호텔,4.8,250000,동대입구역 1번출구 250m,"['주차가능', '야외수영장', '수영장', '와이파이', '사우나', '클럽라운지...","['시설에 비해 가격도 저렴합니당', '너무너무 깨끗하고 쾌적합니다.', '룸서비스...",[],서울특별시 중구 동호로 287,269,169,NaN,NaN,37.560599,127.002136
1,글래드 여의도,4성급,4.7,200640,국회의사당역 4번출구 도보 1분,"['주차가능', '조식운영', '와이파이', '객실금연', '레스토랑', '바', ...","['정말 좋아요 말이 필요 없어요!', '컨디션은 항상 너무 좋네요! 최저가로 저렴...","['데이유즈 쪼매난 글씨에 속아 숙박전환 추가 55천원내고 아주 식겁시키네요', '...",서울특별시 영등포구 의사당대로 16,319,4416,NaN,NaN,37.527409,126.917265
2,글래드 마포,4성급,4.8,143000,공덕역 바로 앞,"['주차가능', '와이파이', '객실금연', '피트니스', '어메니티', '비즈니스']","['너무 좋았습니다! 다음에 재방문할게요~~!', '자주 놀러가는 곳이에요', '완...","['난방이 안되는 호텔춥다고 했더니 체크인시 얘기했는데 어쩌라는 호텔', '지방에 ...",서울특별시 마포구 마포대로 92,378,7717,NaN,NaN,37.542844,126.951336
3,더 리프 사당,일반호텔,4.3,265000,서울특별시 관악구 남현 1길 66 사당역 5번 출구 도보 1분,"['스파/월풀/욕조', '와이파이', '객실금연', '24시간데스크', '비즈니스'...","['너무좋았어용', '너무 친절하시고 시설이 이쁘고 깔끔해서 좋았어요', '기념일 ...",[],서울특별시 관악구 남현1길 66,한국어,77,NaN,NaN,37.475718,126.981124
4,나인트리 프리미어 로카우스 호텔 서울 용산,일반호텔,4.7,158000,용산역 1번 출구 도보 3분,"['주차가능', '조식운영', '와이파이', '객실금연', '레스토랑', '연회장'...","['호캉스하기 좋아요', '좋아용 이용 추천', '어린 아이들이 놀기 좋은 적당한 ...",[],서울특별시 용산구 한강대로23길 25,274,36,NaN,NaN,37.527640,126.964349
